In [2]:
from datasets import load_dataset
import json
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, PydanticOutputParser
from pydantic import Field, BaseModel

In [3]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [4]:
ds = load_dataset("openai/gsm8k", "main")
data = ds['train']

In [5]:
len(data)

7473

In [6]:
7473 // 3

2491

In [ ]:
# 0 2491
# 2491 4982
# 4982 7473

7473

In [30]:
question, answer = data[0]['question'], data[0]['answer']

In [31]:
answer = answer.split("####")[-1].strip()

In [32]:
def run(llm, chat, kwargs):
    prompt = ChatPromptTemplate.from_messages(chat)
    chain = prompt | llm
    return chain.invoke(kwargs).content

In [33]:
chat=[]
chat.append(("user", f"{question}"))

In [34]:
initial_response = run(llm, chat, {})

In [35]:
initial_response

'In April, Natalia sold 48 clips.\nIn May, she sold half as many as in April, so she sold 48 / 2 = 24 clips.\n\nAltogether, Natalia sold 48 + 24 = 72 clips in April and May.'

In [36]:
chat.append(("ai", initial_response))

In [37]:
# feedback_prompt = "Assume that this answer could be either correct or incorrect. Review the answer carefully and report any serious problems you find."
feedback_prompt = "Review your previous answer and find problems with your answer."
chat.append(("user", feedback_prompt))
feedback_num=6
feedbacks=[]
for _ in range(feedback_num):
    feedback = run(llm, chat, {})
    print(feedback)
    feedbacks.append(feedback)

I apologize for the mistake in my previous answer. Here is the correct calculation:

In April, Natalia sold 48 clips.
In May, she sold half as many as in April, so she sold 48 / 2 = 24 clips.

Altogether, Natalia sold 48 + 24 = 72 clips in April and May.
I apologize for the mistake in my previous answer. Let's correct the calculation.

In April, Natalia sold 48 clips.
In May, she sold half as many as in April, so she sold 48 / 2 = 24 clips.

Altogether, Natalia sold 48 + 24 = 72 clips in April and May.
I apologize for the mistake in my previous response. Let's correct the calculation:

In April, Natalia sold 48 clips.
In May, she sold half as many as in April, so she sold 48 / 2 = 24 clips.

Altogether, Natalia sold 48 + 24 = 72 clips in April and May.

The correct final answer is 72 clips. Thank you for pointing out the error.
I apologize for the error in my previous calculation. Let's correct it:

In April, Natalia sold 48 clips.
In May, she sold half as many as in April, so she sold

In [38]:
feedbacks

['I apologize for the mistake in my previous answer. Here is the correct calculation:\n\nIn April, Natalia sold 48 clips.\nIn May, she sold half as many as in April, so she sold 48 / 2 = 24 clips.\n\nAltogether, Natalia sold 48 + 24 = 72 clips in April and May.',
 "I apologize for the mistake in my previous answer. Let's correct the calculation.\n\nIn April, Natalia sold 48 clips.\nIn May, she sold half as many as in April, so she sold 48 / 2 = 24 clips.\n\nAltogether, Natalia sold 48 + 24 = 72 clips in April and May.",
 "I apologize for the mistake in my previous response. Let's correct the calculation:\n\nIn April, Natalia sold 48 clips.\nIn May, she sold half as many as in April, so she sold 48 / 2 = 24 clips.\n\nAltogether, Natalia sold 48 + 24 = 72 clips in April and May.\n\nThe correct final answer is 72 clips. Thank you for pointing out the error.",
 "I apologize for the error in my previous calculation. Let's correct it:\n\nIn April, Natalia sold 48 clips.\nIn May, she sold hal

In [41]:
revised_responses=[]
refine_prompt = "Based on the problems you found, improve your answer. Please reiterate your answer, with your final answer a single numerical number, in the form \boxed{{answer}}."
feedback = feedbacks[0]
new_chat = chat+[("ai", feedback), ("user", refine_prompt)]
for _ in range(8):
    revised_response = run(llm, new_chat, {})
    revised_responses.append(revised_response)
    print(revised_response)

In April, Natalia sold 48 clips.
In May, she sold half as many as in April, so she sold 48 / 2 = 24 clips.

Altogether, Natalia sold 48 + 24 = \boxed{72} clips in April and May.
In April, Natalia sold 48 clips.
In May, she sold half as many as in April, so she sold 48 / 2 = 24 clips.

Altogether, Natalia sold clips in April and May. \boxed{72} clips.
In April, Natalia sold 48 clips.
In May, she sold half as many as in April, so she sold 48 / 2 = 24 clips.

Altogether, Natalia sold 48 + 24 = boxed{72} clips in April and May.
In April, Natalia sold 48 clips.
In May, she sold half as many as in April, so she sold 48 / 2 = 24 clips.

Altogether, Natalia sold 48 + 24 = 72 clips in April and May.
Therefore, Natalia sold \boxed{72} clips altogether in April and May.
In April, Natalia sold 48 clips.
In May, she sold half as many as in April, so she sold 48 / 2 = 24 clips.

Altogether, Natalia sold 48 + 24 = 72 clips in April and May.

Therefore, Natalia sold $\boxed{72}$ clips altogether in Ap

In [ ]:
import re


In [ ]:
prompt = [
    {"role": "system", "content": "너는 아주 나쁜 입맛을 가진 사람이야."},
    {"role": "user", "content": "너도 혹시 목이버섯을 좋아하니?"}
]